In [ ]:
# Import the libraries

import os
import glob
import cv2
import numpy as np
import skimage.util
from skimage import io, img_as_float
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage import exposure

In [ ]:
# Separate the images in folders with their dye names. Example: Dapi images in a folder named Dapi (or dapi),
#       TexaasRed images in a folder named Texased (or texasred, but not texas red) and so on. 
# Run the function below

def folders(names):
    """
    Creates Empty folders for saving contartsed images whose names start with strings in names and end with _Contrast.
    
    Parameters
    ----------
    names: A list containing the names of all the dyes used in the experiment.
    
    Returns
    ----------
    Empty folders in the directory.
    """
    for name in names:
        os.mkdir(name.upper() + "_Contrast")

# First Method Only for DAPI Images (Slow)

In [ ]:
# Going through the raw images in each folder
# Improve the contrast for each image and save the new image in the respective folder with _Contrast suffix

def dapi_slow(raw_img_directory, contrast_img_directory):
    """
    Reads images from a DAPI folder and saves the contrasted ones in a another folder.
    
    Parameters
    ----------
    raw_img_directory: Address of the raw DAPI images with '/' in the end.
    contrast_img_directory: Adrress of the folder where new images will be saved with _Contrast prefix. 
    
    Returns
    ----------
    Contrasted images
    """
    path = raw_img_directory + "/*.*"
    counter = 0
    
    for image in glob.glob(path):
        
        img = img_as_float(io.imread(image))
        sigma_est = np.mean(estimate_sigma(img, multichannel = True))
        patch_kw = dict(patch_size = 5, patch_distance = 6, multichannel = True)
        denoise_img = denoise_nl_means(img, h = 15 * sigma_est, fast_mode = True,
                        ** patch_kw)
        eq_img = exposure.equalize_adapthist(denoise_img)
        final = skimage.util.img_as_ubyte(eq_img)
        new_path = contrast_img_directory + "/" + os.listdir("DAPI")[counter]
        cv2.imwrite(new_path, final)
        counter += 1

# Second Method for All Images (Fast)

In [ ]:
def contrast(raw_img_directory, contrast_img_directory):
    """
    Reads images from any folder and saves the contrasted ones in a another folder.
    
    Parameters
    ----------
    raw_img_directory: Address of the raw images with '/' in the end.
    contrast_img_directory: Adrress of the folder where new images will be saved with _Contrast prefix. 
    
    Returns
    ----------
    Contrasted images
    """
    path = raw_img_directory + "/*.*"
    dye = raw_img_directory.split("/")[0]
    counter = 0
    
    for image in glob.glob(path):
        img = cv2.imread(image, 1)
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        B, G, R = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        cl = clahe.apply(B)
        final = cv2.cvtColor(cv2.merge((cl, G, R)), cv2.COLOR_LAB2BGR)
        new_path = contrast_img_directory + "/" + os.listdir(dye)[counter]
        cv2.imwrite(new_path, final)
        counter += 1